## Take clipped data and prep for export

In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [19]:
# Read in parish council boundaries
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")
gdf = gdf.to_crs("EPSG:27700")
gdf['AreaPC'] = gdf['HECTARES']

In [ ]:
# Interim step, read in all csv's and combine
df_all_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})


for file in file_info_shp.keys():
    df = pd.read_csv(f"data/clippedData/{file}_v1.csv")
    df_all_shp = pd.concat([df_all_shp, df])

In [54]:
# Add area of PC to datasets
df_all_shp = pd.merge(df_all_shp, gdf[['NAME', 'AreaPC']], how = 'left', on = "NAME")
# Add value percent column
def add_percent(row):
    if row['unitName'].lower() == "area":
        val =  (100*row['value'])/row['AreaPC']
    else:
        val =   ""
    return val
    
df_all_shp['valuePercent'] = df_all_shp.apply(add_percent, axis = 1)

In [55]:
# Get rank for each PC on how it compares for each dataset to other PCs
df_all_ranked = pd.DataFrame({'NAME': [], 'value': [],'valuePercent': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [],'mapName': [], 'rank':[], 'rankPercent':[]})

for dataset in df_all_shp.datasetName.unique():
    
    df_dataset = df_all_shp[df_all_shp['datasetName']==dataset]
   # Check if groupColumnValue is null (or na), which means no groups to loop over
    if pd.isnull(df_dataset.groupColumnValue.unique()[0])==True:
        # Sort PC's by value
        df_rank_sorted = df_dataset.sort_values(by = "value", ascending=False)
        # Add rank column which ranks PC's
        df_rank_sorted['rank'] = list(range(1, df_rank_sorted.shape[0]+1))

        # Sort PC's by percentage
        df_rank_percent_sorted = df_rank_sorted.sort_values(by = "valuePercent", ascending=False)
        # Rank PC's by percent
        df_rank_percent_sorted['rankPercent'] = list(range(1, df_rank_percent_sorted.shape[0]+1))
        
        df_all_ranked = pd.concat([df_all_ranked, df_rank_percent_sorted])

    else:
        # If dataset has groupings, need to loop through
        for group in df_dataset.groupColumnValue.unique():
            # Sort PC's by value
            df_rank= df_dataset[df_dataset.groupColumnValue == group]
            df_rank_sorted = df_rank.sort_values(by = "value", ascending=False)
            # Add rank column which ranks PC's
            df_rank_sorted['rank'] = list(range(1, df_rank_sorted.shape[0]+1))

             # Sort PC's by percentage
            df_rank_percent_sorted = df_rank_sorted.sort_values(by = "valuePercent", ascending=False)
            # Rank PC's by percent
            df_rank_percent_sorted['rankPercent'] = list(range(1, df_rank_percent_sorted.shape[0]+1))
            
            df_all_ranked = pd.concat([df_all_ranked, df_rank_percent_sorted])

In [56]:
df_all_ranked['unit'] = df_all_ranked['unit'].replace("HECTARES", "Ha")
df_all_ranked['unit'] = df_all_ranked['unit'].replace("Area (ha)", "Ha")
df_all_ranked['unit'] = df_all_ranked['unit'].replace("METERS", "Metres")
df_all_ranked['unit'] = df_all_ranked['unit'].replace("Count", " ")

In [ ]:
df_all_ranked.to_csv("data/outputClippedData.csv", index=False)